Coding up mass-radius-eccentricty hierarhcial Bayesian model in PyStan.

In [107]:
from __future__ import division, print_function
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["font.size"] = 20
import os
import requests
import pandas as pd
from cStringIO import StringIO
import numpy as np
import pystan


# load data from input files
# TTV amplitudes
inputdata1 = "../data/input/TTVamplitudes.twos.t2.ready.for.jags.3_NAamps_posreal_uncs2.0.txt"
fx1 = open(inputdata1,'r')
datax1 = fx1.read()    
# Get column names
firstlinex1 = datax1.split('\n', 1)[0].split(' ')
firstlinex1 = [w.replace('"', '') for w in firstlinex1]
firstlinex1 = [w.replace('.', '_') for w in firstlinex1]
#print('firstlinex1', firstlinex1)
fx1.close()
# organize data from each file name in list
datax1 = np.genfromtxt(inputdata1, dtype=None, skip_header=1,names=firstlinex1 )

# Outer planet pair data
inputdata2 = "../data/input/Planet_and_Star_data_from_Q1_to_Q12_period_sorting_Pouter_2_sorted.txt"
fx2 = open(inputdata2,'r')
datax2 = fx2.read()    
# Get column names
firstlinex2 = datax2.split('\n', 1)[0].split(' ')
firstlinex2 = [w.replace('"', '') for w in firstlinex2]
firstlinex2 = [w.replace('.', '_') for w in firstlinex2]
#print('firstlinex2', firstlinex2)
fx2.close()
# organize data from each file name in list
datax2 = np.genfromtxt(inputdata2, dtype='float', skip_header=2,names=firstlinex2 )
datax2_s = np.genfromtxt(inputdata2, dtype=None, skip_header=2,names=firstlinex2 )


# Inner planet pair data
inputdata3 = "../data/input/Planet_and_Star_data_from_Q1_to_Q12_period_sorting_Pinner_2_sorted.txt" 
fx3 = open(inputdata2,'r')
datax3 = fx3.read()    
# Get column names
firstlinex3 = datax3.split('\n', 1)[0].split(' ')
firstlinex3 = [w.replace('"', '') for w in firstlinex3]
firstlinex3 = [w.replace('.', '_') for w in firstlinex3]
#print(firstlinex3)
fx3.close()
# organize data from each file name in list
datax3 = np.genfromtxt(inputdata3, dtype='float', skip_header=3,names=firstlinex3 )
datax3_s = np.genfromtxt(inputdata3, dtype=None, skip_header=3,names=firstlinex3 )


#print(datax3['koi_slogg'])
#print(datax2['kepoi_name'])
#print(datax1['Re_V_ext'])
#print('datax1, KOI', datax1['KOI'])

#### Constants
Ndata = len( datax1['KOI'] )

print(Ndata, len( datax2['kepid']), len( datax3['kepid']) )

print(np.hstack(( datax1['KOI'].reshape(-1,1), datax2_s['kepoi_Name'].reshape(-1,1), datax3_s['kepoi_Name'].reshape(-1,1) )))

Nm = 2
P_ext = datax1['P_ext']
P_int = datax1['P_int']
j = datax1['j']
g = datax1['g']
f = datax1['f']
delta = datax1['Delta']
#print( 'delta', delta )
m_sun = 332996.4274 #Msun in Mearth units
r_sun = 110.0
min_radius_s = 0.5 * r_sun #put priors over this
max_radius_s = 2.0 * r_sun  #put priors over this
min_mass_s = 0.8 * m_sun
max_mass_s = 3.0 * m_sun
####

#print(type(datax1))
#print(type(datax2))
#print(type(datax3))

df1 = pd.DataFrame(datax1)
#print(df1)
#print(df1.dtypes)

#df2 = pd.DataFrame(datax2)
#print(df2)
#print(df2.dtypes)

#df3 = pd.DataFrame(datax3)
#print(df3)
#print(df3.dtypes)

    
sigma_radius_s_obs = r_sun * ( ( datax2['koi_srad_err1'] + abs( datax2['koi_srad_err2'] ) ) / 2.0 )
sigma_mass_s_obs = m_sun * ( ( datax2['koi_smass_err1'] + abs( datax2['koi_smass_err2'] ) ) / 2.0 )
sigma_radius_ratio_obs_int = ( datax3['koi_ror_err1'] + abs(datax3['koi_ror_err2'] ) ) / 2.0
sigma_radius_ratio_obs_ext = ( datax2['koi_ror_err1'] + abs(datax2['koi_ror_err2'] ) ) / 2.0
#print('sigma_radius_ratio_obs_ext', sigma_radius_ratio_obs_ext)
#print('sigma_radius_s_obs', sigma_radius_s_obs)
#print('sigma_mass_s_obs', sigma_mass_s_obs)

sigma_va_obs_int = datax1['sig_Re_V_int'] 
sigma_vb_obs_int = datax1['sig_Im_V_int'] 
sigma_va_obs_ext = datax1['sig_Re_V_ext'] 
sigma_vb_obs_ext = datax1['si_Im_V_ext'] 

va_obs_int = datax1['Re_V_int']
va_obs_ext = datax1['Re_V_ext']
vb_obs_int = datax1['Im_V_int']
vb_obs_ext = datax1['Im_V_ext']

#print('radius_s_obs', datax2['koi_srad'] )
#print('radius_s_obs', r_sun * ( datax2['koi_srad'] ))
#print('va_obs_int', va_obs_int)

radius_ratio_obs_int = datax3['koi_ror']
radius_ratio_obs_ext = datax2['koi_ror']
radius_s_obs = r_sun * ( datax2['koi_srad'] )
mass_s_obs = m_sun * ( datax2['koi_smass'] )

#print('radius_s_obs', radius_s_obs)
#print('mass_s_obs', mass_s_obs)


118 118 118
[['222' 'K00222.02' 'K00222.01']
 ['238' 'K00238.02' 'K00238.01']
 ['239' 'K00239.01' 'K00239.02']
 ['244' 'K00244.01' 'K00244.02']
 ['251' 'K00251.02' 'K00251.01']
 ['255' 'K00255.01' 'K00255.02']
 ['266' 'K00266.02' 'K00266.01']
 ['274' 'K00274.02' 'K00274.01']
 ['283' 'K00283.02' 'K00283.01']
 ['295' 'K00295.02' 'K00295.01']
 ['301' 'K00301.02' 'K00301.01']
 ['304' 'K00304.01' 'K00304.02']
 ['312' 'K00312.02' 'K00312.01']
 ['321' 'K00321.02' 'K00321.01']
 ['332' 'K00332.02' 'K00332.01']
 ['341' 'K00341.01' 'K00341.02']
 ['354' 'K00354.01' 'K00354.02']
 ['370' 'K00370.01' 'K00370.02']
 ['413' 'K00413.02' 'K00413.01']
 ['430' 'K00430.01' 'K00430.02']
 ['457' 'K00457.02' 'K00457.01']
 ['508' 'K00508.02' 'K00508.01']
 ['523' 'K00523.01' 'K00523.02']
 ['543' 'K00543.01' 'K00543.02']
 ['546' 'K00546.01' 'K00546.02']
 ['551' 'K00551.01' 'K00551.02']
 ['568' 'K00568.01' 'K00568.02']
 ['572' 'K00572.01' 'K00572.02']
 ['574' 'K00574.01' 'K00574.02']
 ['579' 'K00579.02' 'K00579.01'

In [ ]:

mass_radius_powerlaw_model = """

data {    

    int<lower=1> Nm;
    int<lower=1> Ndata;
    real P_ext[Ndata]; 
    real P_int[Ndata];
    real j[Ndata];
    real g[Ndata];
    real fc[Ndata]; 
    real delta[Ndata]; 
    real radius_ratio_obs_int[Ndata];
    real radius_ratio_obs_ext[Ndata];
    real va_obs_int[Ndata];
    real va_obs_ext[Ndata];
    real vb_obs_int[Ndata];
    real vb_obs_ext[Ndata];
    real sigma_va_obs_int[Ndata];
    real sigma_vb_obs_int[Ndata];
    real sigma_va_obs_ext[Ndata];
    real sigma_vb_obs_ext[Ndata]; 
    real sigma_radius_s[Ndata];
    real sigma_mass_s[Ndata];
    real sigma_radius_ratio_obs_int[Ndata];
    real sigma_radius_ratio_obs_ext[Ndata];
    real radius_s_obs[Ndata];
    real mass_s_obs[Ndata];
    
}

parameters {

    // eccentricity distribution hyperparameters:
    simplex[Nm] f;
    real<lower=0> e_sigma[Nm];
    
    // mass-radius distribution hyperparameters:
    real<lower=0, upper=3> mass_const;
    real<lower=0, upper=2> mass_radius_exp;
    real<lower=0, upper=3> sigma_mass_radius; 
    
    // eccentricity distribution latent variables:
    real<lower=-1,upper=1> h_int[Ndata];
    real<lower=-1,upper=1> k_int[Ndata];
    real<lower=-1,upper=1> h_ext[Ndata];
    real<lower=-1,upper=1> k_ext[Ndata];
    
    // mass-radius distribution latent variables:
    real<lower=0> mass_s_true[Ndata];
    real<lower=0> radius_s_true[Ndata];
    real<lower=0> radius_p_true_int[Ndata]; 
    real<lower=0> mass_p_true_int[Ndata];
    real<lower=0> radius_p_true_ext[Ndata]; 
    real<lower=0> mass_p_true_ext[Ndata];
}

transformed parameters {

    real m_sun;
    real r_sun;
    real min_radius_s;
    real max_radius_s;
    real min_radius_p;
    real max_radius_p;
    real min_mass_s;
    real max_mass_s;
    real C1[Ndata];
    real C2[Ndata];
    real C3[Ndata];
    real va_true_int[Ndata];
    real vb_true_int[Ndata];   
    real va_true_ext[Ndata];
    real vb_true_ext[Ndata];
    real mass_ref_int[Ndata];
    real mass_ref_ext[Ndata];
    real mass_ratio_true_int[Ndata];
    real mass_ratio_true_ext[Ndata];
    real radius_ratio_true_int[Ndata]; 
    real radius_ratio_true_ext[Ndata]; 


    m_sun <- 332996.4274; // Msun in Mearth units
    r_sun <- 110.0;  // Rsun in Rearth units
    
    min_radius_s <- 0.5 * r_sun; // put priors over this
    max_radius_s <- 2.0 * r_sun; // put priors over this
    
    min_radius_p <- 0.1;
    max_radius_p <- 10.0;
    // radius_p_exp ~ uniform(0.0,2.0);
    
    min_mass_s <- 0.8 * m_sun; // put priors over this
    max_mass_s <- 3.0 * m_sun;  // put priors over this
    
    // radius_s_true <- r_sun;
    // mass_s_true <- m_sun;


    for (i in 1:Ndata) {

        // DEFINITIONS //
    
        C1[i] <- P_int[i] / ( ( pi() * ( j[i]^(2./3.) ) * ( (j[i]-1.0)^(1./3.) * delta[i] ) ) );

        C2[i] <- 3.0 / ( 2.0 * delta[i] );
        
        C3[i] <- P_ext[i] / ( pi() * j[i] * delta[i] );
    

        radius_ratio_true_int[i] <- radius_p_true_int[i] / radius_s_true[i];

        mass_ref_int[i] <- mass_const * ( radius_p_true_int[i]^mass_radius_exp );


        radius_ratio_true_ext[i] <- radius_p_true_ext[i] / radius_s_true[i];

        mass_ref_ext[i] <- mass_const * ( radius_p_true_ext[i]^mass_radius_exp );


        mass_ratio_true_int[i] <- mass_p_true_int[i] / mass_s_true[i];
        
        mass_ratio_true_ext[i] <- mass_p_true_ext[i] / mass_s_true[i];

        va_true_int[i] <- ( (-1.0) * C1[i] * mass_ratio_true_ext[i] * fc[i] ) - ( ( C1[i] * C2[i] * mass_ratio_true_ext[i] ) * ( ( fc[i] * h_int[i] ) + ( g[i] * h_ext[i] ) ) );

        vb_true_int[i] <- ( C1[i] * C2[i] * mass_ratio_true_ext[i] ) * ( ( fc[i] * k_int[i] ) + ( g[i] * k_ext[i] ) );

        va_true_ext[i] <- ( (-1.0) * C3[i] * mass_ratio_true_int[i] * g[i] ) + ( ( C2[i] * C3[i] * mass_ratio_true_int[i] ) * ( ( fc[i] * h_int[i] ) + ( g[i] * h_ext[i] ) ) );

        vb_true_ext[i] <- ( (-1.0) * C2[i] * C3[i] * mass_ratio_true_int[i] ) * ( ( fc[i] * k_int[i] ) + ( g[i] * k_ext[i] ) );

    }
}

model {
        
    real ps_h_int[Nm];
    real ps_k_int[Nm];
    real ps_h_ext[Nm];
    real ps_k_ext[Nm];
 
    
    // eccentricity distribution model section (generalize for interior/exterior pairs):
    
    e_sigma ~ uniform(0, 1.0);
    

    // interior planet eccentricty:
    
    for (n in 1:Ndata)
      for (jj in 1:Nm) {
        ps_h_int[jj] <- log(f[jj]) + normal_log(h_int[n],0.0,e_sigma[jj]);
      }
      increment_log_prob(log_sum_exp(ps_h_int));

    for (n in 1:Ndata)
      for (jj in 1:Nm) {
        ps_k_int[jj] <- log(f[jj]) + normal_log(k_int[n],0.0,e_sigma[jj]);
      }
      increment_log_prob(log_sum_exp(ps_k_int));
      
      
      //exterior planet eccentricity:
      
    for (n in 1:Ndata)
      for (jj in 1:Nm) {
        ps_h_ext[jj] <- log(f[jj]) + normal_log(h_ext[n],0.0,e_sigma[jj]);
      }
      increment_log_prob(log_sum_exp(ps_h_ext));
      
    for (n in 1:Ndata)
      for (jj in 1:Nm) {
        ps_k_ext[jj] <- log(f[jj]) + normal_log(k_ext[n],0.0,e_sigma[jj]);
      }
      increment_log_prob(log_sum_exp(ps_k_ext));


    // mass-radius distribution model section:
    
    mass_const ~ uniform(0.0,3.0);
    
    mass_radius_exp ~ uniform(0.0,2.0);
    
    sigma_mass_radius ~ uniform(0.0,3.0); 
    
    
    
    for (i in 1:Ndata) {
    
        // DISTRIBUTIONS //
        
	
	    radius_s_true[i] ~ uniform( min_radius_s, max_radius_s );
        
        radius_s_obs[i] ~ normal( radius_s_true[i], 1.0 / sigma_radius_s[i] ) T[0,]; 


        mass_s_true[i] ~ uniform( min_mass_s, max_mass_s );
        
        mass_s_obs[i] ~ normal( mass_s_true[i], 1.0 / sigma_mass_s[i] );

        
        radius_p_true_int[i] ~ uniform( min_radius_p, max_radius_p ); 
        
        radius_ratio_obs_int[i] ~ normal( radius_ratio_true_int[i], 1.0 / sigma_radius_ratio_obs_int[i] );

        
        mass_p_true_int[i] ~ normal( mass_ref_int[i], 1.0 / sigma_mass_radius ) T[0,];

        
        va_obs_int[i] ~ normal( va_true_int[i], 1.0 / sigma_va_obs_int[i] );
        
        vb_obs_int[i] ~ normal( vb_true_int[i], 1.0/ sigma_vb_obs_int[i] );   


        radius_p_true_ext[i] ~ uniform( min_radius_p, max_radius_p ); 

        radius_ratio_obs_ext[i] ~ normal( radius_ratio_true_ext[i], 1.0 / sigma_radius_ratio_obs_ext[i] ); 

        
        mass_p_true_ext[i] ~ normal( mass_ref_ext[i], 1.0 / sigma_mass_radius ) T[0,];

  
        va_obs_ext[i] ~ normal( va_true_ext[i], 1.0 / sigma_va_obs_ext[i] );
        
        vb_obs_ext[i] ~ normal( vb_true_ext[i], 1.0 / sigma_vb_obs_ext[i] ); 
        
    }


}

"""

data = {'P_ext':P_ext, 'P_int':P_int, 'j':j, 'g':g, 'fc':f, 'delta':delta, 
        'radius_ratio_obs_int':radius_ratio_obs_int, 'radius_ratio_obs_ext':radius_ratio_obs_ext,
        'va_obs_int':va_obs_int, 'va_obs_ext':va_obs_ext, 'vb_obs_int':vb_obs_int,
        'vb_obs_ext':vb_obs_ext, 'sigma_va_obs_int':sigma_va_obs_int,
        'sigma_vb_obs_int':sigma_vb_obs_int, 'sigma_va_obs_ext':sigma_va_obs_ext,
        'sigma_vb_obs_ext':sigma_vb_obs_ext, 'sigma_radius_s':sigma_radius_s_obs,
        'sigma_mass_s':sigma_mass_s_obs, 'sigma_radius_ratio_obs_int':sigma_radius_ratio_obs_int, 
        'sigma_radius_ratio_obs_ext':sigma_radius_ratio_obs_ext, 'radius_s_obs':radius_s_obs, 
        'mass_s_obs':mass_s_obs, 'Ndata':Ndata, 'Nm':Nm}


#sm = StanModel(model_code=mass_radius_powerlaw_model)
#fit = sm.sampling(data=data, iter=10, chains=2, init=init, n_jobs=-1)

fit = pystan.stan(model_code=mass_radius_powerlaw_model, data=data, iter=1, chains=1, n_jobs=-1);

la = fit.extract(permuted=True)  # return a dictionary of arrays
alpha = la['mass_radius_exp']
beta = la['mass_const']
lnf0 = la['sigma_mass_radius']
print(mass_radius_exp)
print(mass_const)
print(sigma_mass_radius)

a = fit.extract(permuted=False)
print(a)
print(fit)

print(mass_radius_exp)

plt.hist(mass_radius_exp)
fit.plot()


